In [1]:
# 0909d -1-1-1
# epoch 长一点70->100
# thrd 0.01 -> 0.005
# 改为 pytorch lightning
# pyg 172->201
# 0917a 512 256 128
# 0918a 改为128 128 128 +mlp
# 发散

In [2]:
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_6c1',
     'total_volume_mean_0c1',
     'total_volume_mean_1c1', 
     'total_volume_mean_3c1',
     'total_volume_mean_4c1', 
     'total_volume_mean_6c1',
     'trade_size_mean_0c1',
     'trade_size_mean_1c1', 
     'trade_size_mean_3c1',
     'trade_size_mean_4c1', 
     'trade_size_mean_6c1',
     'trade_order_count_mean_0c1',
     'trade_order_count_mean_1c1',
     'trade_order_count_mean_3c1',
     'trade_order_count_mean_4c1',
     'trade_order_count_mean_6c1',      
     'price_spread_mean_0c1',
     'price_spread_mean_1c1',
     'price_spread_mean_3c1',
     'price_spread_mean_4c1',
     'price_spread_mean_6c1',   
     'bid_spread_mean_0c1',
     'bid_spread_mean_1c1',
     'bid_spread_mean_3c1',
     'bid_spread_mean_4c1',
     'bid_spread_mean_6c1',       
     'ask_spread_mean_0c1',
     'ask_spread_mean_1c1',
     'ask_spread_mean_3c1',
     'ask_spread_mean_4c1',
     'ask_spread_mean_6c1',   
     'volume_imbalance_mean_0c1',
     'volume_imbalance_mean_1c1',
     'volume_imbalance_mean_3c1',
     'volume_imbalance_mean_4c1',
     'volume_imbalance_mean_6c1',       
     'bid_ask_spread_mean_0c1',
     'bid_ask_spread_mean_1c1',
     'bid_ask_spread_mean_3c1',
     'bid_ask_spread_mean_4c1',
     'bid_ask_spread_mean_6c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_6c1'] 

In [3]:
!pip install ../input/pyg201whl/*.whl

Processing /kaggle/input/pyg201whl/googledrivedownloader-0.4-py2.py3-none-any.whl
Processing /kaggle/input/pyg201whl/isodate-0.6.0-py2.py3-none-any.whl
Processing /kaggle/input/pyg201whl/rdflib-6.0.0-py3-none-any.whl
Processing /kaggle/input/pyg201whl/torch_cluster-1.5.9-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/pyg201whl/torch_geometric-2.0.1-py3-none-any.whl
Processing /kaggle/input/pyg201whl/torch_scatter-2.0.8-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/pyg201whl/torch_sparse-0.6.11-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/pyg201whl/torch_spline_conv-1.2.1-cp37-cp37m-linux_x86_64.whl


In [4]:
'''机器学习区'''
import numpy as np
import numpy.matlib
import pandas as pd
from sklearn.model_selection import GroupKFold,KFold
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.cluster import KMeans

'''绘图区'''
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")

'''pytroch'''
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import ModuleList, BatchNorm1d
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from torch import Tensor
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything

from pytorch_lightning.callbacks import EarlyStopping
'''pyg'''
import torch_geometric
from torch_geometric.datasets import Reddit
from torch_geometric.data import Data,NeighborSampler
from torch_geometric.data import NeighborSampler as RawNeighborSampler
if torch_geometric.__version__ != '1.7.2':
    from torch_geometric.loader.neighbor_sampler import EdgeIndex
else:
    from torch_geometric.data.sampler import EdgeIndex
from torch_geometric.nn import GCNConv,GATConv, ChebConv,SAGEConv

'''系统资源区'''
from tqdm.notebook import tqdm
from typing import Optional, List, NamedTuple
import multiprocessing
from joblib import Parallel, delayed
from glob import glob
import warnings
warnings.filterwarnings('ignore')
import pickle
import gc
from torch_geometric.nn import TransformerConv
heads = 4

In [5]:
DEBUG = 0
THRD = 0.02

In [6]:
BASE = '../input/optiver-realized-volatility-prediction/'
EBASE = '../input/optiver-clf/'
EBASE1 = '../input/opt-0924/'
BOOK_TST = BASE+'book_test.parquet/'
BOOK_TRN = BASE+'book_train.parquet/'
TRADE_TST = BASE+'trade_test.parquet/'
TRADE_TRN = BASE+'trade_train.parquet/'
TRN = BASE+'train.csv'
TST = BASE+'test.csv'

if DEBUG == 1:
    TST = TRN
    BOOK_TST = BOOK_TRN
    
    trn = pd.read_csv(TRN)
    trn['row_id'] = trn['stock_id'].astype(str) + '-' + trn['time_id'].astype(str)
    tst = trn.copy()
    y_true = tst['target']
    tst.drop('target',axis=1,inplace =True)
else:
    trn = pd.read_csv(TRN)
    trn['row_id'] = trn['stock_id'].astype(str) + '-' + trn['time_id'].astype(str)
    tst = pd.read_csv(TST)
print('trn',trn.shape,'tst',tst.shape)

trn (428932, 4) tst (3, 3)


In [7]:
trn_time_id = trn.time_id.unique()
all_stock_id = trn.stock_id.unique()

In [8]:
def load_book_data_by_stock_id(stock_id,is_train=True):
    if is_train == True:
        df = pd.read_parquet(f'{BOOK_TRN}stock_id={stock_id}')
    else:
        df = pd.read_parquet(f'{BOOK_TST}stock_id={stock_id}')
    return df

def book_feature_by_stock_id_for_trn(stock_id):
    return book_feature_by_stock_id(stock_id)

def book_feature_by_stock_id_for_tst(stock_id):
    return book_feature_by_stock_id(stock_id,is_train=False)

def calc_wap1(df):
    return (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])

def calc_price(df):
    diff = abs(df[['bid_price1','ask_price1','bid_price2','ask_price2']].diff())
    min_diff = np.nanmin(diff.where(lambda x: x > 0))
    n_ticks = (diff / min_diff).round()
    scale = 0.01 / np.nanmean(diff / n_ticks)
    return scale

def calc_prices(stock_id,is_train):
    try:
        book = load_book_data_by_stock_id(stock_id,is_train)
    except:
        return pd.DataFrame()
    book['wap1'] = calc_wap1(book)
    df = book.groupby('time_id').apply(calc_price).to_frame('price').reset_index()
    df['stock_id'] = stock_id
    df['first_wap'] = df['price'] * book.groupby('time_id')['wap1'].first().values
    df['last_wap'] = df['price'] * book.groupby('time_id')['wap1'].last().values
    df['mean_wap'] = df['price'] * book.groupby('time_id')['wap1'].mean().values
    return df

In [9]:
stock_ids = trn.stock_id.unique()

In [10]:
SEED=42

def set_all_seed(SEED=42):
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    torch.backends.cudnn.deterministic=True
    torch.backends.cudnn.benchmark = False
    seed_everything(SEED)
set_all_seed(SEED)

In [11]:
def get_cluster_knn_values(out_train):
    np.random.seed(SEED)
    out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')

    # out_train[out_train.isna().any(axis=1)]
    out_train = out_train.fillna(out_train.mean())
    out_train.head()

    # Code to add the just the read data after first execution

    # Data separation based on knn ++
    nfolds = 5 # number of folds
    index = []
    totDist = []
    values = []

    # Generates a matriz with the values of 
    mat = out_train.values
    scaler = MinMaxScaler(feature_range=(-1, 1))
    mat = scaler.fit_transform(mat)
    nind = int(mat.shape[0]/nfolds) # number of individuals

    # Adds index in the last column
    mat = np.c_[mat,np.arange(mat.shape[0])]
    lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)
    lineNumber = np.sort(lineNumber)[::-1]
    for n in range(nfolds):
        totDist.append(np.zeros(mat.shape[0]-nfolds))

    # Saves index
    for n in range(nfolds):    
        values.append([lineNumber[n]])

    s=[]
    for n in range(nfolds):
        s.append(mat[lineNumber[n],:])
        mat = np.delete(mat, obj=lineNumber[n], axis=0)

    for n in range(nind-1):    
        luck = np.random.uniform(0,1,nfolds)

        for cycle in range(nfolds):
            # Saves the values of index           
            s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)
            sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
            totDist[cycle] += sumDist        

            # Probabilities
            f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totDist
            j = 0
            kn = 0
            for val in f:
                j += val        
                if (j > luck[cycle]): # the column was selected
                    break
                kn +=1
            lineNumber[cycle] = kn

            # Delete line of the value added    
            for n_iter in range(nfolds):
                totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
                j= 0

            s[cycle] = mat[lineNumber[cycle],:]
            values[cycle].append(int(mat[lineNumber[cycle],-1]))
            mat = np.delete(mat, obj=lineNumber[cycle], axis=0)

    for n_mod in range(nfolds):
        values[n_mod] = out_train.index[values[n_mod]]    
    return values 
    
knn_values = get_cluster_knn_values(trn)

In [12]:
def get_cluster_labels(train_p):
    train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')
    corr = train_p.corr()
    ids = corr.index
    kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)
    print(kmeans.labels_)
    l = []
    for n in range(7):
        l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    return l

cluster_labels = get_cluster_labels(trn)

[1 0 4 2 1 1 2 4 6 2 1 0 4 4 1 1 1 2 4 4 4 0 1 1 3 1 1 4 3 4 3 4 4 1 3 3 4
 3 4 1 4 1 4 4 1 0 4 4 1 0 0 3 3 3 2 0 2 4 1 4 4 1 4 1 0 3 3 0 3 0 6 5 3 3
 0 1 2 0 3 3 3 4 1 1 0 2 3 3 1 0 1 4 4 4 4 4 1 3 1 0 1 4 1 0 1 4 1 0 4 0 4
 0]


# node

In [13]:
from IPython.core.display import display, HTML

import glob
import os
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib

import plotly.express as px
import plotly.graph_objects as go

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans

from numpy.random import seed
seed(42)

import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
from keras import backend as K
from keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation

import warnings
warnings.filterwarnings('ignore')

In [14]:
def read_train_test():
    # Function to read our base train and test set
    
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')

    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    
    return train, test

# Read train and test
train, test = read_train_test()

Our training set has 428932 rows


In [15]:
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'

def calc_wap1(df):
    # Function to calculate first WAP
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap2(df):
    # Function to calculate second WAP
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def log_return(series):
    # Function to calculate the log of the return
    return np.log(series).diff()

def realized_volatility(series):
    # Calculate the realized volatility
    return np.sqrt(np.sum(series**2))

def count_unique(series):
    # Function to count unique elements of a series
    return len(np.unique(series))

def book_preprocessor(file_path):
    # Function to preprocess book data (for each stock id)
    
    df = pd.read_parquet(file_path)
    
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'price_spread2':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std],
        "bid_ask_spread":[np.sum, np.mean, np.std],
    }
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Function to get group stats for different windows (seconds in bucket)
        
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    
    # Merge all
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')

    # Drop unnecesary time_ids
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200'], axis = 1, inplace = True)
    
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    
    return df_feature


def trade_preprocessor(file_path):
    # Function to preprocess trade data (for each stock id)
    
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count':[np.mean,np.sum,np.max],
    }
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Function to get group stats for different windows (seconds in bucket)
        
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    # Merge all
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')

    # Drop unnecesary time_ids
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200','time_id'], axis = 1, inplace = True)
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    
    return df_feature


def get_time_stock(df):
    # Function to get group stats for the stock_id and time_id
    
    # Get realized volatility columns
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    
    return df
    
    
def preprocessor(list_stock_ids, is_train = True):
    # Funtion to make preprocessing function in parallel (for each stock id)
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    
    return df


def rmspe(y_true, y_pred):
    # Function to calculate the root mean squared percentage error
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def feval_rmspe(y_pred, lgb_train):
    # Function to early stop with root mean squared percentage error
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [16]:
# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()

# Preprocess them using Parallel and our single stock id functions
#train_ = preprocessor(train_stock_ids, is_train = True)
#train = train.merge(train_, on = ['row_id'], how = 'left')

# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()

# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
#train = get_time_stock(train)
test = get_time_stock(test)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.3s finished


In [17]:
# replace by order sum (tau)
#train['size_tau'] = np.sqrt(1/train['trade_seconds_in_bucket_count_unique'])
test['size_tau'] = np.sqrt(1/test['trade_seconds_in_bucket_count_unique'])
#train['size_tau_400'] = np.sqrt(1/train['trade_seconds_in_bucket_count_unique_400'])
test['size_tau_400'] = np.sqrt(1/test['trade_seconds_in_bucket_count_unique_400'])
#train['size_tau_300'] = np.sqrt(1/train['trade_seconds_in_bucket_count_unique_300'])
test['size_tau_300'] = np.sqrt(1/test['trade_seconds_in_bucket_count_unique_300'])
#train['size_tau_200'] = np.sqrt(1/train['trade_seconds_in_bucket_count_unique_200'])
test['size_tau_200'] = np.sqrt(1/test['trade_seconds_in_bucket_count_unique_200'])

In [18]:
# tau2 
#train['size_tau2'] = np.sqrt(1/train['trade_order_count_sum'])
test['size_tau2'] = np.sqrt(1/test['trade_order_count_sum'])
#train['size_tau2_400'] = np.sqrt(0.25/train['trade_order_count_sum'])
test['size_tau2_400'] = np.sqrt(0.25/test['trade_order_count_sum'])
#train['size_tau2_300'] = np.sqrt(0.5/train['trade_order_count_sum'])
test['size_tau2_300'] = np.sqrt(0.5/test['trade_order_count_sum'])
#train['size_tau2_200'] = np.sqrt(0.75/train['trade_order_count_sum'])
test['size_tau2_200'] = np.sqrt(0.75/test['trade_order_count_sum'])

# delta tau
#train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']
test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

In [19]:
trn_df = pd.read_parquet('../input/opt-0924/train0923.parq')
tst_df = test.copy()

In [20]:
num_col = trn_df.columns.difference(['time_id','target','row_id','stock_id']).tolist()
num_col = [col for col in num_col if '_stock' not in col ]
cat_col = 'stock_id'
fea_col = [cat_col]+num_col
print(len(fea_col),len(num_col),cat_col)
print(np.nanmean(trn_df[num_col],axis=0).round(5).tolist())

265 264 stock_id
[-0.0002, -0.0002, -0.0002, -0.0002, 0.00012, 0.00012, 0.00011, 0.00011, -0.0764, -0.04995, -0.03724, -0.02469, 0.0004, 0.00039, 0.00039, 0.00039, 0.00017, 0.00016, 0.00016, 0.00016, 0.15191, 0.09935, 0.07406, 0.04911, 0.0002, 0.0002, 0.0002, 0.00019, 0.00012, 0.00011, 0.00011, 0.00011, 0.07551, 0.0494, 0.03682, 0.02442, -0.0, 0.0, 0.0, 0.0, 0.00423, 0.00334, 0.01001, 0.00334, 0.00109, 0.00153, 0.00285, 0.0087, 0.00285, 0.00091, 0.00133, 0.0023, 0.0072, 0.0023, 0.0007, 0.0011, 0.01244, 0.00423, 0.00139, 0.0019, 0.00023, 0.00022, 0.00022, 0.00022, -1e-05, 0.0, 1e-05, 1e-05, -0.0, 0.0, 0.0, 0.0, 0.00581, 0.00458, 0.01428, 0.00458, 0.00123, 0.00226, 0.00392, 0.01241, 0.00392, 0.00102, 0.00196, 0.00315, 0.01023, 0.00315, 0.00077, 0.00162, 0.01766, 0.00581, 0.00158, 0.00281, 0.00032, 0.00031, 0.0003, 0.0003, -1e-05, 0.0, 1e-05, 1e-05, 0.00106, 0.00104, 0.00103, 0.00103, 0.00027, 0.00024, 0.00024, 0.00023, 0.37455, 0.24301, 0.18061, 0.11947, 0.00066, 0.00065, 0.00064, 0.0006

In [21]:
'''先qt'''
trn_df.replace([np.inf, -np.inf], np.nan,inplace=True)
tst_df.replace([np.inf, -np.inf], np.nan,inplace=True)
qt_train = []

for col in num_col:
    qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')
    trn_df[col] = qt.fit_transform(trn_df[[col]])
    tst_df[col] = qt.transform(tst_df[[col]])    
    qt_train.append(qt)


In [22]:
print(np.nanmean(trn_df[num_col],axis=0).round(5).tolist())

[-0.00397, -0.00413, -0.00399, -0.00444, -0.71527, -0.77696, -0.81845, -0.88174, -0.00395, -0.00424, -0.00421, -0.00444, 0.00366, 0.00393, 0.00369, 0.00401, -0.62097, -0.66829, -0.7003, -0.74899, 0.0038, 0.00414, 0.00408, 0.00412, 0.00341, 0.0035, 0.0033, 0.00316, -0.71579, -0.77775, -0.82183, -0.88707, 0.00347, 0.00366, 0.00367, 0.0035, -0.00166, -0.00119, -0.00197, -0.00244, 0.00146, 0.00199, 0.00484, 0.00282, 0.00204, 0.00371, 0.00181, 0.00437, 0.00282, 0.00081, 0.00363, 0.00138, 0.00398, 0.00282, -0.00038, 0.0035, 0.00431, 0.00272, 0.0021, 0.00374, 0.00145, 0.00184, 0.0016, 0.00122, -0.00148, -0.0018, -0.00291, -0.00256, -0.00184, -0.00063, -0.00306, -0.00258, 0.00148, 0.0017, 0.00382, 0.00281, 0.00274, 0.00401, 0.00174, 0.00321, 0.00288, 0.00164, 0.00398, 0.00146, 0.00278, 0.00299, 0.00121, 0.00392, 0.00368, 0.00283, 0.00193, 0.00396, 0.00132, 0.00144, 0.00138, 0.0012, -0.00145, -0.00108, -0.0036, -0.00264, 0.002, 0.00221, 0.00222, 0.00228, -0.0001, -0.00362, -0.0074, -0.01767, 0.

In [23]:
def get_agg_fea(trn_df,tst_df,cluster_labels):
    mat = []
    matTest = []
    n = 0
    for ind in cluster_labels:
        print(ind)
        newDf = trn_df.loc[trn_df['stock_id'].isin(ind) ]
        newDf = newDf.groupby(['time_id']).agg(np.nanmean)
        newDf.loc[:,'stock_id'] = str(n)+'c1'
        mat.append ( newDf )
        newDf = tst_df.loc[tst_df['stock_id'].isin(ind) ]    
        newDf = newDf.groupby(['time_id']).agg(np.nanmean)
        newDf.loc[:,'stock_id'] = str(n)+'c1'
        matTest.append ( newDf )
        n+=1

    mat1 = pd.concat(mat).reset_index()
    mat1.drop(columns=['target'],inplace=True)
    mat2 = pd.concat(matTest).reset_index()

    '''变换形状，改名'''
    mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])#下面是left join
    mat1 = mat1.pivot(index='time_id', columns='stock_id')
    mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
    mat1.reset_index(inplace=True)

    mat2 = mat2.pivot(index='time_id', columns='stock_id')
    mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
    mat2.reset_index(inplace=True)    
    
    '''拼接'''
    trn_df = pd.merge(trn_df,mat1[nnn].set_index('time_id'),how='left',left_on='time_id' ,right_index=True)
    tst_df = pd.merge(tst_df,mat2[nnn].set_index('time_id'),how='left',left_on='time_id' ,right_index=True)
    return trn_df,tst_df

In [24]:
trn_df,tst_df = get_agg_fea(trn_df,tst_df,cluster_labels)#主要是agg和scaler部分

[1, 11, 22, 50, 55, 56, 62, 73, 76, 78, 84, 87, 96, 101, 112, 116, 122, 124, 126]
[0, 4, 5, 10, 15, 16, 17, 23, 26, 28, 29, 36, 42, 44, 48, 53, 66, 69, 72, 85, 94, 95, 100, 102, 109, 111, 113, 115, 118, 120]
[3, 6, 9, 18, 61, 63, 86, 97]
[27, 31, 33, 37, 38, 40, 58, 59, 60, 74, 75, 77, 82, 83, 88, 89, 90, 98, 99, 110]
[2, 7, 13, 14, 19, 20, 21, 30, 32, 34, 35, 39, 41, 43, 46, 47, 51, 52, 64, 67, 68, 70, 93, 103, 104, 105, 107, 108, 114, 119, 123, 125]
[81]
[8, 80]


In [25]:
prices_df1 = pd.read_parquet(EBASE+'/df_prices1.pq') #明示数据的价格矩阵
prices_df2 = pd.concat(Parallel(n_jobs=4, verbose=51)(delayed(calc_prices)(r,False) for r in all_stock_id))
prices_df = pd.concat([prices_df1,prices_df2]).reset_index(drop=True)
print(f"出现time_id:{prices_df.time_id.nunique()}")
trn_df = trn_df.merge(prices_df[['time_id','stock_id','price']],how='left')
tst_df = tst_df.merge(prices_df[['time_id','stock_id','price']],how='left')

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:    1.6s
[Parallel(

In [26]:
num_col = trn_df.columns.difference(['time_id','target','row_id','stock_id','price']).tolist()
num_col = [col for col in num_col if '_stock' not in col ]
cat_col = 'stock_id'
fea_col = [cat_col]+num_col
print(len(fea_col),len(num_col),cat_col)
print(np.nanmean(trn_df[num_col],axis=0).round(5).tolist())

315 314 stock_id
[-0.00397, 0.26565, 0.24725, -0.00413, -0.00399, -0.48296, -0.00444, 0.01801, -0.38396, -0.71527, -0.77696, -0.81845, -0.88174, -0.00395, -0.00424, -0.00421, -0.00444, 0.00366, -0.27904, -0.26055, 0.00393, 0.00369, 0.51924, 0.00401, -0.03044, 0.38682, -0.62097, -0.66829, -0.7003, -0.74899, 0.0038, 0.00414, 0.00408, 0.00412, 0.00341, -0.27585, -0.25061, 0.0035, 0.0033, 0.48332, 0.00316, -0.01064, 0.39158, -0.71579, -0.77775, -0.82183, -0.88707, 0.00347, 0.00366, 0.00367, 0.0035, -0.00166, -0.00119, -0.00197, -0.00244, 0.00146, 0.32488, -0.07157, 0.00199, 0.00484, 0.00282, 0.00204, 0.00371, 0.00181, 0.00437, 0.00282, 0.00081, 0.00363, 0.26177, 0.00138, 0.00398, 0.00282, -0.00038, 0.0035, -0.47825, 0.64281, 0.00431, 0.00272, 0.0021, 0.00374, 0.00145, 0.00184, 0.0016, 0.00122, -0.00148, -0.0018, -0.00291, -0.00256, -0.00184, -0.00063, -0.00306, -0.00258, 0.00148, 0.0017, 0.00382, 0.00281, 0.00274, 0.00401, 0.00174, 0.00321, 0.00288, 0.00164, 0.00398, 0.00146, 0.00278, 0.00

In [27]:
trn_df[num_col] = trn_df[num_col].fillna(trn_df[num_col].mean())
tst_df[num_col] = tst_df[num_col].fillna(trn_df[num_col].mean())
scaler = MinMaxScaler(feature_range=(-1, 1))
gg = 50
for i in range(len(num_col)//gg+1):
    col = num_col[i*gg:i*gg+gg]
    trn_df[col] = scaler.fit_transform(trn_df[col])
    tst_df[col] = scaler.transform(tst_df[col])
node_df = pd.concat([trn_df,tst_df]).drop_duplicates().reset_index(drop=True)
print(np.nanmean(trn_df[num_col],axis=0).round(5).tolist())

[-0.00076, 0.38125, 0.47259, -0.00079, -0.00077, 0.3754, -0.00085, 0.56466, 0.02654, -0.13757, -0.14943, -0.15742, -0.16959, -0.00076, -0.00082, -0.00081, -0.00085, 0.0007, -0.43483, -0.50416, 0.00076, 0.00071, -0.35503, 0.00077, -0.57381, -0.01111, -0.11943, -0.12853, -0.13469, -0.14406, 0.00073, 0.0008, 0.00078, 0.00079, 0.00066, -0.45508, -0.49316, 0.00067, 0.00063, -0.36022, 0.00061, -0.59866, 0.03119, -0.13767, -0.14959, -0.15806, -0.17061, 0.00067, 0.0007, 0.00071, 0.00067, -0.00032, -0.00023, -0.00038, -0.00047, 0.00028, -0.17547, -0.23273, 0.00038, 0.00093, 0.00054, 0.00039, 0.00071, 0.00035, 0.00084, 0.00054, 0.00016, 0.0007, -0.29056, 0.00027, 0.00077, 0.00054, -7e-05, 0.00067, -0.30045, 0.07329, 0.00083, 0.00052, 0.0004, 0.00072, 0.00028, 0.00035, 0.00031, 0.00024, -0.00029, -0.00035, -0.00056, -0.00049, -0.00035, -0.00012, -0.00059, -0.0005, 0.00029, 0.00033, 0.00074, 0.00054, 0.00053, 0.00077, 0.00033, 0.00062, 0.00055, 0.00032, 0.00076, 0.00028, 0.00053, 0.00058, 0.00023,

# edge

In [28]:
# prices_df1 = pd.read_parquet(EBASE+'/df_prices1.pq') #明示数据的价格矩阵
# prices_df2 = pd.concat(Parallel(n_jobs=4, verbose=51)(delayed(calc_prices)(r,False) for r in stock_ids)) #在线上是test部分的价格矩阵
# prices_df = pd.concat([prices_df1,prices_df2]).reset_index(drop=True)
# print(f"出现time_id:{prices_df.time_id.nunique()}")

In [29]:
'''每个time_id的起始价格和结束价格的query矩阵'''
first_df = prices_df[['time_id', 'stock_id', 'first_wap']].pivot('time_id', 'stock_id', 'first_wap')
last_df = prices_df[['time_id', 'stock_id', 'last_wap']].pivot('time_id', 'stock_id', 'last_wap')
first_df = first_df.fillna(first_df.mean())
last_df = last_df.fillna(first_df.mean())

In [30]:
def get_nearby_time_id(query_df,template_df):
    def get_nearby_time_id_(time_id):
        template_df_=template_df.drop(time_id,axis=1)
        query = query_df[time_id].values.repeat(len(template_df_.columns)).reshape(-1,len(template_df_.columns))
        diffs = np.square((template_df_- query)/query).sum(axis=0)
        diffs = diffs.sort_values()[:2].reset_index().rename(columns = {'time_id':'tid',0:'loss'})
        diffs['time_id'] = time_id
        return diffs    
    edge_df = pd.concat(Parallel(n_jobs=4, verbose=1)(delayed(get_nearby_time_id_)(time_id) for time_id in tqdm(query_df.columns)))
    edge_df = edge_df[['time_id','tid','loss']]
    edge_df .columns = ['a','b','w']
    return edge_df

In [31]:
'''每个time_id分别按照收尾两头各找5个，然后做无向图且去重，边的权重取出现过的最小值'''
last_edge_df = get_nearby_time_id(last_df.T,first_df.T)
first_edge_df = get_nearby_time_id(first_df.T,last_df.T)
# last_edge_df = pd.read_parquet(EBASE1+'last_edge_df.pq')
# first_edge_df = pd.read_parquet(EBASE1+'first_edge_df.pq')
edge_df = pd.concat([first_edge_df,last_edge_df])
#edge_df = pd.concat([edge_df,edge_df.rename(columns={'a':'b','b':'a'})])
edge_df = edge_df.groupby(['a','b'])['w'].min().to_frame('w').reset_index().rename(columns={'a':'b','b':'a'})

  0%|          | 0/3831 [00:00<?, ?it/s]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    8.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   15.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:   25.5s
[Parallel(n_jobs=4)]: Done 1640 tasks      | elapsed:   46.4s
[Parallel(n_jobs=4)]: Done 2740 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 3824 out of 3831 | elapsed:  1.6min remaining:    0.2s
[Parallel(n_jobs=4)]: Done 3831 out of 3831 | elapsed:  1.6min finished


  0%|          | 0/3831 [00:00<?, ?it/s]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  76 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 376 tasks      | elapsed:    9.1s
[Parallel(n_jobs=4)]: Done 876 tasks      | elapsed:   21.2s
[Parallel(n_jobs=4)]: Done 1576 tasks      | elapsed:   38.3s
[Parallel(n_jobs=4)]: Done 2476 tasks      | elapsed:   59.7s
[Parallel(n_jobs=4)]: Done 3576 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 3831 out of 3831 | elapsed:  1.5min finished


In [32]:
edge_df.shape

(10119, 3)

In [33]:
'''阈值截断'''
edge_df = edge_df.query('w < @THRD').reset_index(drop = True)
edge_df.shape

(9768, 3)

In [34]:
edge_df.shape

(9768, 3)

In [35]:
'''拓展到所有stock'''
edge_dfs = []
for stock_id in all_stock_id:
    edge_df_ = edge_df.copy()
    edge_df_['a'] = str(stock_id)+'-' +edge_df.a.astype(str)
    edge_df_['b'] = str(stock_id)+'-' +edge_df.b.astype(str)
    edge_dfs.append(edge_df_)
edge_df = pd.concat(edge_dfs).reset_index(drop=True)
edge_df

,b,a,w
0,0-5,0-1205,0.001942
1,0-5,0-26708,0.001290
2,0-5,0-30183,0.000688
3,0-11,0-2811,0.000387
4,0-11,0-29583,0.000537
...,...,...,...
1094011,126-32763,126-14749,0.006277
1094012,126-32763,126-23737,0.001415
1094013,126-32763,126-26471,0.007328
1094014,126-32767,126-1962,0.000532


In [36]:
'''孤立点'''
isolate_node = list(set(node_df['row_id']) - set(edge_df['a'].unique()))
isolate_edge_df = pd.DataFrame({'a':isolate_node,'b':isolate_node,'w':0})
edge_df = pd.concat([edge_df,isolate_edge_df]).reset_index(drop=True)

'''重新编码'''
row_id_index_dict = dict(zip(node_df.row_id,node_df.index))

edge_df['a'] = edge_df['a'].map(row_id_index_dict)
edge_df['b'] = edge_df['b'].map(row_id_index_dict)

'''构造边使用了笛卡尔积，但是有些点不存在'''
edge_df = edge_df[~edge_df.a.isna() & ~edge_df.b.isna()]
edge_df.shape

(1112214, 3)

In [37]:
class MyFLod:
    def __init__(self,time_id_list):
        self.time_id_list=time_id_list
        
    def __iter__(self):
        self.flod = 0
        self.n_folds = len(self.time_id_list)
        return self
    
    def __next__(self):
        if self.flod >= self.n_folds:
            raise StopIteration
        time_ids = np.arange(self.n_folds).astype(int)    
        time_ids = np.delete(time_ids,obj=self.flod, axis=0) 
        val_time_id = list(self.time_id_list[self.flod])
        trn_time_id = \
        list(self.time_id_list[time_ids[0]])+\
        list(self.time_id_list[time_ids[1]])+\
        list(self.time_id_list[time_ids[2]])+\
        list(self.time_id_list[time_ids[3]])
        self.flod += 1
        return trn_time_id,val_time_id
            

knn_flod = MyFLod(knn_values)

# GNN

In [38]:
class GS_CFG():
    save_path = 'gs.pth' #模型路径
    lr = 0.0005 #学习率
    label_col = 'fraud'
    batch_size = 512  # 每批的样本数量
    epoch = 80  # 遍历多少遍样本集
    verbose = 1
    output_num = 1
    fea_num = len(num_col)
    stock_embedding_size = 24

In [39]:
if len(tst) == 3:
    GS_CFG.epoch=2

In [40]:
def RMSPELoss(y_pred, y_true):
    return torch.sqrt(torch.mean(((y_true - y_pred) / y_true) ** 2 ))
def rmspe(y_true, y_pred):
    # Function to calculate the root mean squared percentage error
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

In [41]:
class Batch(NamedTuple):
    x: Tensor
    y: Tensor
    adjs: List[EdgeIndex]

class OptGNNDM(pl.LightningDataModule):
    def __init__(self, train_mask,val_mask,test_mask):
        super().__init__()
        self.x = torch.FloatTensor(node_df[['stock_id']+num_col].values)
        self.y = torch.tensor(node_df['target']).reshape(-1,1)
        self.edge_index = torch.LongTensor(edge_df.drop_duplicates(['a','b'])[['a','b']].values).t()
        self.train_mask = train_mask
        self.val_mask = val_mask
        self.test_mask = test_mask
        
    def train_dataloader(self):
        return NeighborSampler(self.edge_index, node_idx=self.train_mask,
                               sizes=[-1,-1,-1], return_e_id=True,
                               transform=self.convert_batch, batch_size=GS_CFG.batch_size,
                               num_workers=8, shuffle=True)

    def val_dataloader(self):
        return NeighborSampler(self.edge_index, node_idx=self.val_mask,
                               sizes=[-1,-1,-1], return_e_id=True,
                               transform=self.convert_batch, batch_size=GS_CFG.batch_size,
                               num_workers=4, shuffle=False)

    def test_dataloader(self):
        return NeighborSampler(self.edge_index, node_idx=self.test_mask,
                               sizes=[-1,-1,-1], return_e_id=True,
                               transform=self.convert_batch, batch_size=GS_CFG.batch_size,
                               num_workers=4, shuffle=False)

    def convert_batch(self, batch_size, n_id, adjs):
        return Batch(
            x=self.x[n_id],
            y=self.y[n_id[:batch_size]],
            adjs=adjs,
        )

In [42]:
class GraphSAGE(pl.LightningModule):
    def __init__(self, in_channels: int, out_channels: int, hidden_channels: int = 128,num_layers=3):
        super().__init__()
        #self.save_hyperparameters()
        self.num_layers = num_layers
        self.hidden_channels = hidden_channels
        
        hidden_channels = self.hidden_channels
        self.convs = ModuleList()
        self.convs.append(TransformerConv(in_channels, hidden_channels//heads,heads=heads))
        self.convs.append(TransformerConv(hidden_channels, hidden_channels//heads,heads=heads))
        self.convs.append(TransformerConv(hidden_channels, hidden_channels//heads,heads=heads))
        self.embedding = nn.Embedding(127, GS_CFG.stock_embedding_size)
        self.Lin1 = nn.Linear(hidden_channels,out_channels)
                 
    def forward(self, x: Tensor, adjs: List[EdgeIndex]) -> Tensor:
        cat_data = x[:,0].long()
        num_data = x[:,1:]
        x = torch.cat([self.embedding(cat_data),num_data],axis=1)
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            x = F.relu(x)
        x = self.Lin1(x)
        return x

    def training_step(self, batch: Batch, batch_idx: int):
        x, y, adjs = batch
        out = model(x, adjs)
        loss = RMSPELoss(out, y)
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
        return loss
    
    def validation_step(self, batch: Batch, batch_idx: int):
        x, y, adjs = batch
        y_pred = model(x, adjs)
        
        return {'y_pred':y_pred,'y':y}
        #loss = RMSPELoss(out, y)
        #self.log("valid_loss", loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)

    def validation_epoch_end(self, outputs):
        y_pred = torch.cat([out['y_pred'] for out in outputs], dim=0)
        y = torch.cat([out['y'] for out in outputs], dim=0)
        loss = RMSPELoss(y_pred,y)
        self.log("valid_loss", loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
        return {'dummy_item': 0} 
    
    def test_step(self, batch: Batch, batch_idx: int):
        x, y, adjs = batch
        y_pred = model(x, adjs)
        return {'y_pred': y_pred}
    
    def test_epoch_end(self, outputs):
        y_pred = torch.cat([out['y_pred'] for out in outputs], dim=0)
        y_pred = y_pred.detach().cpu().numpy().reshape(-1)
        self.test_y_pred = y_pred  # Save prediction internally for easy access
        # We need to return something 
        return {'dummy_item': 0} 
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=GS_CFG.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 3,threshold=0.001,  factor = 0.5, verbose =True)
        return {
           'optimizer': optimizer,
           'lr_scheduler': scheduler, # Changed scheduler to lr_scheduler
           'monitor': 'valid_loss'
       }

In [43]:
import gc
del trn_df
gc.collect()

20

In [44]:
edge_index = torch.LongTensor(edge_df.drop_duplicates(['a','b'])[['a','b']].values).t()
x = torch.FloatTensor(node_df[['stock_id']+num_col].values)

In [45]:
model_list = pickle.load(open(EBASE1+"0928b_model_list25", "rb"))

In [46]:
tst_preds = 0

In [47]:
model_list_ = model_list[0:5]
n_folds = 5
scores = []
tst_pred = 0

print(edge_index.shape)
for n_count,(trn_time_id,val_time_id) in enumerate(iter(knn_flod)):
    flod = n_count+1
    print('CV {}/{}'.format(flod, n_folds))
    
    train_mask = torch.tensor(node_df.time_id.isin(trn_time_id))
    val_mask = torch.tensor(node_df.time_id.isin(val_time_id))
    test_mask = ~(train_mask | val_mask)
    opt_gnn_dm = OptGNNDM(train_mask,val_mask,test_mask)

    model = GraphSAGE(GS_CFG.fea_num+GS_CFG.stock_embedding_size, 
                      GS_CFG.output_num)
    model.load_state_dict(model_list_[n_count])
    trainer = Trainer(gpus=1)
    trainer.test(model, opt_gnn_dm.test_dataloader())
    tst_pred += model.test_y_pred.reshape(1,-1)[0].clip(0,0.09)
    scores.append(trainer.validate(model, opt_gnn_dm.val_dataloader())[0]['valid_loss'])
    del trainer,model,opt_gnn_dm
    gc.collect()
print(np.array(scores).round(5),'cv',np.mean(scores).round(5))
print(tst_pred/flod)
tst_preds += tst_pred/flod

torch.Size([2, 1112214])
CV 1/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.19748344938999188}
--------------------------------------------------------------------------------
CV 2/5


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20136738891421377}
--------------------------------------------------------------------------------
CV 5/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20321101419399554}
--------------------------------------------------------------------------------
[0.19748 0.2141  0.19933 0.20137 0.20321] cv 0.2031
[0.00090436 0.00258131 0.00258131]


In [48]:
model_list_ = model_list[5:10]
n_folds = 5
scores = []
tst_pred = 0

print(edge_index.shape)
for n_count,(trn_time_id,val_time_id) in enumerate(iter(knn_flod)):
    flod = n_count+1
    print('CV {}/{}'.format(flod, n_folds))
    
    train_mask = torch.tensor(node_df.time_id.isin(trn_time_id))
    val_mask = torch.tensor(node_df.time_id.isin(val_time_id))
    test_mask = ~(train_mask | val_mask)
    opt_gnn_dm = OptGNNDM(train_mask,val_mask,test_mask)

    model = GraphSAGE(GS_CFG.fea_num+GS_CFG.stock_embedding_size, 
                      GS_CFG.output_num)
    model.load_state_dict(model_list_[n_count])
    trainer = Trainer(gpus=1)
    trainer.test(model, opt_gnn_dm.test_dataloader())
    tst_pred += model.test_y_pred.reshape(1,-1)[0].clip(0,0.09)
    scores.append(trainer.validate(model, opt_gnn_dm.val_dataloader())[0]['valid_loss'])
    del trainer,model,opt_gnn_dm
    gc.collect()
print(np.array(scores).round(5),'cv',np.mean(scores).round(5))
print(tst_pred/flod)
tst_preds += tst_pred/flod

torch.Size([2, 1112214])
CV 1/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.1980906019663457}
--------------------------------------------------------------------------------
CV 2/5


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20184460667343265}
--------------------------------------------------------------------------------
CV 5/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20471798786053724}
--------------------------------------------------------------------------------
[0.19809 0.22638 0.19888 0.20184 0.20472] cv 0.20598
[0.00116878 0.00269201 0.00269201]


In [49]:
model_list_ = model_list[10:15]
n_folds = 5
scores = []
tst_pred = 0

print(edge_index.shape)
for n_count,(trn_time_id,val_time_id) in enumerate(iter(knn_flod)):
    flod = n_count+1
    print('CV {}/{}'.format(flod, n_folds))
    
    train_mask = torch.tensor(node_df.time_id.isin(trn_time_id))
    val_mask = torch.tensor(node_df.time_id.isin(val_time_id))
    test_mask = ~(train_mask | val_mask)
    opt_gnn_dm = OptGNNDM(train_mask,val_mask,test_mask)

    model = GraphSAGE(GS_CFG.fea_num+GS_CFG.stock_embedding_size, 
                      GS_CFG.output_num)
    model.load_state_dict(model_list_[n_count])
    trainer = Trainer(gpus=1)
    trainer.test(model, opt_gnn_dm.test_dataloader())
    tst_pred += model.test_y_pred.reshape(1,-1)[0].clip(0,0.09)
    scores.append(trainer.validate(model, opt_gnn_dm.val_dataloader())[0]['valid_loss'])
    del trainer,model,opt_gnn_dm
    gc.collect()
print(np.array(scores).round(5),'cv',np.mean(scores).round(5))
print(tst_pred/flod)
tst_preds += tst_pred/flod

torch.Size([2, 1112214])
CV 1/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.19748454894347162}
--------------------------------------------------------------------------------
CV 2/5


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.2018901091924895}
--------------------------------------------------------------------------------
CV 5/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20342311271324656}
--------------------------------------------------------------------------------
[0.19748 0.21372 0.19879 0.20189 0.20342] cv 0.20306
[0.00099937 0.00268279 0.00268279]


In [50]:
model_list_ = model_list[15:20]
n_folds = 5
scores = []
tst_pred = 0

print(edge_index.shape)
for n_count,(trn_time_id,val_time_id) in enumerate(iter(knn_flod)):
    flod = n_count+1
    print('CV {}/{}'.format(flod, n_folds))
    
    train_mask = torch.tensor(node_df.time_id.isin(trn_time_id))
    val_mask = torch.tensor(node_df.time_id.isin(val_time_id))
    test_mask = ~(train_mask | val_mask)
    opt_gnn_dm = OptGNNDM(train_mask,val_mask,test_mask)

    model = GraphSAGE(GS_CFG.fea_num+GS_CFG.stock_embedding_size, 
                      GS_CFG.output_num)
    model.load_state_dict(model_list_[n_count])
    trainer = Trainer(gpus=1)
    trainer.test(model, opt_gnn_dm.test_dataloader())
    tst_pred += model.test_y_pred.reshape(1,-1)[0].clip(0,0.09)
    scores.append(trainer.validate(model, opt_gnn_dm.val_dataloader())[0]['valid_loss'])
    del trainer,model,opt_gnn_dm
    gc.collect()
print(np.array(scores).round(5),'cv',np.mean(scores).round(5))
print(tst_pred/flod)
tst_preds += tst_pred/flod

torch.Size([2, 1112214])
CV 1/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.19759408701288708}
--------------------------------------------------------------------------------
CV 2/5


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20102570870153977}
--------------------------------------------------------------------------------
CV 5/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20283219465063682}
--------------------------------------------------------------------------------
[0.19759 0.20555 0.19801 0.20103 0.20283] cv 0.201
[0.00112072 0.00259797 0.00259797]


In [51]:
model_list_ = model_list[20:25]
n_folds = 5
scores = []
tst_pred = 0

print(edge_index.shape)
for n_count,(trn_time_id,val_time_id) in enumerate(iter(knn_flod)):
    flod = n_count+1
    print('CV {}/{}'.format(flod, n_folds))
    
    train_mask = torch.tensor(node_df.time_id.isin(trn_time_id))
    val_mask = torch.tensor(node_df.time_id.isin(val_time_id))
    test_mask = ~(train_mask | val_mask)
    opt_gnn_dm = OptGNNDM(train_mask,val_mask,test_mask)

    model = GraphSAGE(GS_CFG.fea_num+GS_CFG.stock_embedding_size, 
                      GS_CFG.output_num)
    model.load_state_dict(model_list_[n_count])
    trainer = Trainer(gpus=1)
    trainer.test(model, opt_gnn_dm.test_dataloader())
    tst_pred += model.test_y_pred.reshape(1,-1)[0].clip(0,0.09)
    scores.append(trainer.validate(model, opt_gnn_dm.val_dataloader())[0]['valid_loss'])
    del trainer,model,opt_gnn_dm
    gc.collect()
print(np.array(scores).round(5),'cv',np.mean(scores).round(5))
print(tst_pred/flod)
tst_preds += tst_pred/flod

torch.Size([2, 1112214])
CV 1/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.1989800140276078}
--------------------------------------------------------------------------------
CV 2/5


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20353318217277502}
--------------------------------------------------------------------------------
CV 5/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.2021865635485438}
--------------------------------------------------------------------------------
[0.19898 0.21645 0.19849 0.20353 0.20219] cv 0.20393
[0.00111854 0.00264386 0.00264386]


In [52]:
tst_preds = tst_preds/(len(model_list)//5)
tst_preds

array([0.00106236, 0.00263959, 0.00263959], dtype=float32)

In [53]:
tst_preds1 = tst_preds

# edge2

In [54]:
'''每个time_id的起始价格和结束价格的query矩阵'''
first_df = prices_df[['time_id', 'stock_id', 'first_wap']].pivot('time_id', 'stock_id', 'first_wap')
last_df = prices_df[['time_id', 'stock_id', 'last_wap']].pivot('time_id', 'stock_id', 'last_wap')
first_df = first_df.fillna(first_df.mean())
last_df = last_df.fillna(first_df.mean())

In [55]:
def get_nearby_time_id(query_df,template_df):
    def get_nearby_time_id_(time_id):
        template_df_=template_df.drop(time_id,axis=1)
        query = query_df[time_id].values.repeat(len(template_df_.columns)).reshape(-1,len(template_df_.columns))
        diffs = np.square((template_df_- query)/query).sum(axis=0)
        diffs = diffs.sort_values()[:3].reset_index().rename(columns = {'time_id':'tid',0:'loss'})
        diffs['time_id'] = time_id
        return diffs    
    edge_df = pd.concat(Parallel(n_jobs=4, verbose=1)(delayed(get_nearby_time_id_)(time_id) for time_id in tqdm(query_df.columns)))
    edge_df = edge_df[['time_id','tid','loss']]
    edge_df .columns = ['a','b','w']
    return edge_df

In [56]:
'''每个time_id分别按照收尾两头各找5个，然后做无向图且去重，边的权重取出现过的最小值'''
last_edge_df = get_nearby_time_id(last_df.T,first_df.T)
first_edge_df = get_nearby_time_id(first_df.T,last_df.T)
# last_edge_df = pd.read_parquet(EBASE1+'last_edge_df.pq')
# first_edge_df = pd.read_parquet(EBASE1+'first_edge_df.pq')
edge_df = pd.concat([first_edge_df,last_edge_df])
#edge_df = pd.concat([edge_df,edge_df.rename(columns={'a':'b','b':'a'})])
edge_df = edge_df.groupby(['a','b'])['w'].min().to_frame('w').reset_index().rename(columns={'a':'b','b':'a'})

  0%|          | 0/3831 [00:00<?, ?it/s]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.7s
[Parallel(n_jobs=4)]: Done 304 tasks      | elapsed:    9.9s
[Parallel(n_jobs=4)]: Done 804 tasks      | elapsed:   21.6s
[Parallel(n_jobs=4)]: Done 1504 tasks      | elapsed:   39.0s
[Parallel(n_jobs=4)]: Done 2404 tasks      | elapsed:  1.0min
[Parallel(n_jobs=4)]: Done 3504 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done 3824 out of 3831 | elapsed:  1.6min remaining:    0.2s
[Parallel(n_jobs=4)]: Done 3831 out of 3831 | elapsed:  1.6min finished


  0%|          | 0/3831 [00:00<?, ?it/s]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  76 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 376 tasks      | elapsed:    9.1s
[Parallel(n_jobs=4)]: Done 876 tasks      | elapsed:   20.9s
[Parallel(n_jobs=4)]: Done 1576 tasks      | elapsed:   38.3s
[Parallel(n_jobs=4)]: Done 2476 tasks      | elapsed:   59.6s
[Parallel(n_jobs=4)]: Done 3576 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 3824 out of 3831 | elapsed:  1.5min remaining:    0.2s
[Parallel(n_jobs=4)]: Done 3831 out of 3831 | elapsed:  1.5min finished


In [57]:
edge_df.shape

(14130, 3)

In [58]:
'''阈值截断'''
edge_df = edge_df.query('w < @THRD').reset_index(drop = True)
edge_df.shape

(13477, 3)

In [59]:
'''拓展到所有stock'''
edge_dfs = []
for stock_id in all_stock_id:
    edge_df_ = edge_df.copy()
    edge_df_['a'] = str(stock_id)+'-' +edge_df.a.astype(str)
    edge_df_['b'] = str(stock_id)+'-' +edge_df.b.astype(str)
    edge_dfs.append(edge_df_)
edge_df = pd.concat(edge_dfs).reset_index(drop=True)
edge_df

,b,a,w
0,0-5,0-1205,0.001942
1,0-5,0-26708,0.001290
2,0-5,0-30183,0.000688
3,0-5,0-31471,0.001543
4,0-11,0-2811,0.000387
...,...,...,...
1509419,126-32763,126-26471,0.007328
1509420,126-32767,126-1962,0.000532
1509421,126-32767,126-6545,0.001968
1509422,126-32767,126-10734,0.001645


In [60]:
'''孤立点'''
isolate_node = list(set(node_df['row_id']) - set(edge_df['a'].unique()))
isolate_edge_df = pd.DataFrame({'a':isolate_node,'b':isolate_node,'w':0})
edge_df = pd.concat([edge_df,isolate_edge_df]).reset_index(drop=True)

'''重新编码'''
row_id_index_dict = dict(zip(node_df.row_id,node_df.index))

edge_df['a'] = edge_df['a'].map(row_id_index_dict)
edge_df['b'] = edge_df['b'].map(row_id_index_dict)

'''构造边使用了笛卡尔积，但是有些点不存在'''
edge_df = edge_df[~edge_df.a.isna() & ~edge_df.b.isna()]
edge_df.shape

(1522114, 3)

In [61]:
class MyFLod:
    def __init__(self,time_id_list):
        self.time_id_list=time_id_list
        
    def __iter__(self):
        self.flod = 0
        self.n_folds = len(self.time_id_list)
        return self
    
    def __next__(self):
        if self.flod >= self.n_folds:
            raise StopIteration
        time_ids = np.arange(self.n_folds).astype(int)    
        time_ids = np.delete(time_ids,obj=self.flod, axis=0) 
        val_time_id = list(self.time_id_list[self.flod])
        trn_time_id = \
        list(self.time_id_list[time_ids[0]])+\
        list(self.time_id_list[time_ids[1]])+\
        list(self.time_id_list[time_ids[2]])+\
        list(self.time_id_list[time_ids[3]])
        self.flod += 1
        return trn_time_id,val_time_id
            

knn_flod = MyFLod(knn_values)

# GNN2

In [62]:
class GS_CFG():
    save_path = 'gs.pth' #模型路径
    lr = 0.0005 #学习率
    label_col = 'fraud'
    batch_size = 512  # 每批的样本数量
    epoch = 80  # 遍历多少遍样本集
    verbose = 1
    output_num = 1
    fea_num = len(num_col)
    stock_embedding_size = 24

In [63]:
model_list = pickle.load(open(EBASE1+"0928_model_list25", "rb"))

In [64]:
tst_preds = 0

In [65]:
model_list_ = model_list[0:5]
n_folds = 5
scores = []
tst_pred = 0

print(edge_index.shape)
for n_count,(trn_time_id,val_time_id) in enumerate(iter(knn_flod)):
    flod = n_count+1
    print('CV {}/{}'.format(flod, n_folds))
    
    train_mask = torch.tensor(node_df.time_id.isin(trn_time_id))
    val_mask = torch.tensor(node_df.time_id.isin(val_time_id))
    test_mask = ~(train_mask | val_mask)
    opt_gnn_dm = OptGNNDM(train_mask,val_mask,test_mask)

    model = GraphSAGE(GS_CFG.fea_num+GS_CFG.stock_embedding_size, 
                      GS_CFG.output_num)
    model.load_state_dict(model_list_[n_count])
    trainer = Trainer(gpus=1)
    trainer.test(model, opt_gnn_dm.test_dataloader())
    tst_pred += model.test_y_pred.reshape(1,-1)[0].clip(0,0.09)
    scores.append(trainer.validate(model, opt_gnn_dm.val_dataloader())[0]['valid_loss'])
    del trainer,model,opt_gnn_dm
    gc.collect()
print(np.array(scores).round(5),'cv',np.mean(scores).round(5))
print(tst_pred/flod)
tst_preds += tst_pred/flod

torch.Size([2, 1112214])
CV 1/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.19766837274237947}
--------------------------------------------------------------------------------
CV 2/5


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20217740076966678}
--------------------------------------------------------------------------------
CV 5/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20209369448939107}
--------------------------------------------------------------------------------
[0.19767 0.23247 0.19881 0.20218 0.20209] cv 0.20664
[0.00139622 0.00265015 0.00265015]


In [66]:
model_list_ = model_list[5:10]
n_folds = 5
scores = []
tst_pred = 0

print(edge_index.shape)
for n_count,(trn_time_id,val_time_id) in enumerate(iter(knn_flod)):
    flod = n_count+1
    print('CV {}/{}'.format(flod, n_folds))
    
    train_mask = torch.tensor(node_df.time_id.isin(trn_time_id))
    val_mask = torch.tensor(node_df.time_id.isin(val_time_id))
    test_mask = ~(train_mask | val_mask)
    opt_gnn_dm = OptGNNDM(train_mask,val_mask,test_mask)

    model = GraphSAGE(GS_CFG.fea_num+GS_CFG.stock_embedding_size, 
                      GS_CFG.output_num)
    model.load_state_dict(model_list_[n_count])
    trainer = Trainer(gpus=1)
    trainer.test(model, opt_gnn_dm.test_dataloader())
    tst_pred += model.test_y_pred.reshape(1,-1)[0].clip(0,0.09)
    scores.append(trainer.validate(model, opt_gnn_dm.val_dataloader())[0]['valid_loss'])
    del trainer,model,opt_gnn_dm
    gc.collect()
print(np.array(scores).round(5),'cv',np.mean(scores).round(5))
print(tst_pred/flod)
tst_preds += tst_pred/flod

torch.Size([2, 1112214])
CV 1/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.1980669557520919}
--------------------------------------------------------------------------------
CV 2/5


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20208677427090585}
--------------------------------------------------------------------------------
CV 5/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20245291125185733}
--------------------------------------------------------------------------------
[0.19807 0.22799 0.19721 0.20209 0.20245] cv 0.20556
[0.00109122 0.00268485 0.00268485]


In [67]:
model_list_ = model_list[10:15]
n_folds = 5
scores = []
tst_pred = 0

print(edge_index.shape)
for n_count,(trn_time_id,val_time_id) in enumerate(iter(knn_flod)):
    flod = n_count+1
    print('CV {}/{}'.format(flod, n_folds))
    
    train_mask = torch.tensor(node_df.time_id.isin(trn_time_id))
    val_mask = torch.tensor(node_df.time_id.isin(val_time_id))
    test_mask = ~(train_mask | val_mask)
    opt_gnn_dm = OptGNNDM(train_mask,val_mask,test_mask)

    model = GraphSAGE(GS_CFG.fea_num+GS_CFG.stock_embedding_size, 
                      GS_CFG.output_num)
    model.load_state_dict(model_list_[n_count])
    trainer = Trainer(gpus=1)
    trainer.test(model, opt_gnn_dm.test_dataloader())
    tst_pred += model.test_y_pred.reshape(1,-1)[0].clip(0,0.09)
    scores.append(trainer.validate(model, opt_gnn_dm.val_dataloader())[0]['valid_loss'])
    del trainer,model,opt_gnn_dm
    gc.collect()
print(np.array(scores).round(5),'cv',np.mean(scores).round(5))
print(tst_pred/flod)
tst_preds += tst_pred/flod

torch.Size([2, 1112214])
CV 1/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.19853320610582484}
--------------------------------------------------------------------------------
CV 2/5


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20270530910788728}
--------------------------------------------------------------------------------
CV 5/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.2034030225737812}
--------------------------------------------------------------------------------
[0.19853 0.21765 0.19697 0.20271 0.2034 ] cv 0.20385
[0.00097466 0.00263579 0.00263579]


In [68]:
model_list_ = model_list[15:20]
n_folds = 5
scores = []
tst_pred = 0

print(edge_index.shape)
for n_count,(trn_time_id,val_time_id) in enumerate(iter(knn_flod)):
    flod = n_count+1
    print('CV {}/{}'.format(flod, n_folds))
    
    train_mask = torch.tensor(node_df.time_id.isin(trn_time_id))
    val_mask = torch.tensor(node_df.time_id.isin(val_time_id))
    test_mask = ~(train_mask | val_mask)
    opt_gnn_dm = OptGNNDM(train_mask,val_mask,test_mask)

    model = GraphSAGE(GS_CFG.fea_num+GS_CFG.stock_embedding_size, 
                      GS_CFG.output_num)
    model.load_state_dict(model_list_[n_count])
    trainer = Trainer(gpus=1)
    trainer.test(model, opt_gnn_dm.test_dataloader())
    tst_pred += model.test_y_pred.reshape(1,-1)[0].clip(0,0.09)
    scores.append(trainer.validate(model, opt_gnn_dm.val_dataloader())[0]['valid_loss'])
    del trainer,model,opt_gnn_dm
    gc.collect()
print(np.array(scores).round(5),'cv',np.mean(scores).round(5))
print(tst_pred/flod)
tst_preds += tst_pred/flod

torch.Size([2, 1112214])
CV 1/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.19871681327133714}
--------------------------------------------------------------------------------
CV 2/5


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20141787326149385}
--------------------------------------------------------------------------------
CV 5/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20139504448645915}
--------------------------------------------------------------------------------
[0.19872 0.20207 0.1963  0.20142 0.2014 ] cv 0.19998
[0.00081649 0.00258665 0.00258665]


In [69]:
model_list_ = model_list[20:25]
n_folds = 5
scores = []
tst_pred = 0

print(edge_index.shape)
for n_count,(trn_time_id,val_time_id) in enumerate(iter(knn_flod)):
    flod = n_count+1
    print('CV {}/{}'.format(flod, n_folds))
    
    train_mask = torch.tensor(node_df.time_id.isin(trn_time_id))
    val_mask = torch.tensor(node_df.time_id.isin(val_time_id))
    test_mask = ~(train_mask | val_mask)
    opt_gnn_dm = OptGNNDM(train_mask,val_mask,test_mask)

    model = GraphSAGE(GS_CFG.fea_num+GS_CFG.stock_embedding_size, 
                      GS_CFG.output_num)
    model.load_state_dict(model_list_[n_count])
    trainer = Trainer(gpus=1)
    trainer.test(model, opt_gnn_dm.test_dataloader())
    tst_pred += model.test_y_pred.reshape(1,-1)[0].clip(0,0.09)
    scores.append(trainer.validate(model, opt_gnn_dm.val_dataloader())[0]['valid_loss'])
    del trainer,model,opt_gnn_dm
    gc.collect()
print(np.array(scores).round(5),'cv',np.mean(scores).round(5))
print(tst_pred/flod)
tst_preds += tst_pred/flod

torch.Size([2, 1112214])
CV 1/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.1978636807782871}
--------------------------------------------------------------------------------
CV 2/5


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20052852756986436}
--------------------------------------------------------------------------------
CV 5/5


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'valid_loss': 0.20303704754791557}
--------------------------------------------------------------------------------
[0.19786 0.20136 0.1974  0.20053 0.20304] cv 0.20004
[0.00105768 0.00269166 0.00269166]


In [70]:
tst_preds = tst_preds/(len(model_list)//5)
tst_preds

array([0.00106725, 0.00264982, 0.00264982], dtype=float32)

In [71]:
tst['target'] = tst_preds1*0.4+tst_preds*0.6
tst[['row_id','target']].to_csv('submission.csv', index=False)
tst.head()

,stock_id,time_id,row_id,target
0,0,4,0-4,0.001065
1,0,32,0-32,0.002646
2,0,34,0-34,0.002646


In [72]:
import sys as sys
object_v_li = []
object_size_li = []
for object_v in dir():
    
    getsizeof_str = 'sys.getsizeof('+ object_v +')'
    try:
        size = eval(getsizeof_str)
    except NameError as e:
        print('except:', e)
        continue
        
    #print(object_v)
    object_v_li.append(object_v)
    object_size_li.append(size)
    
dddd = pd.DataFrame({'object_name':object_v_li,'size':pd.Series(object_size_li)/1024/1024})
dddd.sort_values('size',ascending = False).head(10)

,object_name,size
208,node_df,1219.333514
61,_59,198.053631
55,_35,143.542984
168,edge_df,46.451263
258,trn,36.319337
255,train,36.319337
223,prices_df1,22.907471
234,row_id_index_dict,20.000099
222,prices_df,19.635147
196,last_df,3.302818
